Steps:
 - Download video from YouTube
 - Transcript video using openai whisper
 - Combine segments for a more meaningful chunks
 - Text embed transcript segments
 - Text embed question
 - Calculate similarity

In [23]:
!pip install openai openai-whisper transformers pytube tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import whisper
model = whisper.load_model('base')

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 105MiB/s]


In [24]:
from pytube import YouTube
youtube_video_url = "https://www.youtube.com/watch?v=kSu7fNDKqvM"
youtube_video = YouTube(youtube_video_url)
audio = youtube_video.streams.get_audio_only().download()


/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:113: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
transcript_output = model.transcribe(audio)

In [25]:
transcript_output

{'text': " So Catholics have the Vatican. Soccer fans, old Trapperton Manchester. And for car guys like me, Jay Leno's garage. Jay, thanks again for bringing us here. Thank you guys for wondering where they're going. Catholics have the Vatican. I go, oh, what is it? It's totally different. I am in the wrong, oh, I see. But you guys see, you make you comprise. Yeah, you know. That's why you live in the Vatican. You know, you're in the Vatican. You know, you're in the Vatican. You know, you're in the Vatican. You're in the Vatican. So Catholics have the Vatican. Soccer fans, old Trapperton Manchester. And for car guys like me, Jay Leno's garage. Jay, thanks again for bringing us here. Thank you guys for wondering where they're going. Catholics have the Vatican. I go, oh, what is it? It's totally different. I am in the wrong, oh, I see. But you guys see, you make you comprise. Yeah, you know. That's why you live in the Vatican. You know, you're in the Vatican. You know, you're in the Vati

In [41]:
transcript_data = []
for segment in transcript_output['segments']:
        transcript_data.append({
                "id": f"{youtube_video_url}&t={segment['start']}s",
                "text": segment["text"].strip(),
                "start": segment['start'],
                "end": segment['end']
        })


[{'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=0.0s',
  'text': 'So Catholics have the Vatican.',
  'start': 0.0,
  'end': 7.0},
 {'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=7.0s',
  'text': 'Soccer fans, old Trapperton Manchester.',
  'start': 7.0,
  'end': 10.0},
 {'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=10.0s',
  'text': "And for car guys like me, Jay Leno's garage.",
  'start': 10.0,
  'end': 13.0},
 {'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=13.0s',
  'text': 'Jay, thanks again for bringing us here.',
  'start': 13.0,
  'end': 15.0},
 {'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=15.0s',
  'text': "Thank you guys for wondering where they're going.",
  'start': 15.0,
  'end': 17.0},
 {'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=17.0s',
  'text': 'Catholics have the Vatican.',
  'start': 17.0,
  'end': 18.0},
 {'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=18.0s',
  'text': 'I go, oh, what is it?',
  'start': 18.0,
  

In [ ]:
len(transcript_data)

In [ ]:
transcript_data[0:5]

In [63]:
# combining each 6 (window) transcript segments into one, with overlabbing of 3 
# segments =(stride), these number might need fine tunnig to obtain the most 
# meaningful semgentation
transcript_new_data = []
window = 6  # number of segments to combine
stride = 3  # number of segments to 'stride' over, used to create overlap
for i in (range(0, len(transcript_data), stride)):
    i_end = min(len(transcript_data)-1, i+window)
    text = ' '.join(_['text'] for _ in transcript_data[i:i_end])
    transcript_new_data.append({
        'start': transcript_data[i]['start'],
        'end': transcript_data[i_end]['end'],
        'text': text,
        'id': transcript_data[i]['id'],
    })


In [69]:
len(transcript_new_data)

161

In [68]:
transcript_new_data[0:5]

[{'start': 0.0,
  'end': 19.0,
  'text': "So Catholics have the Vatican. Soccer fans, old Trapperton Manchester. And for car guys like me, Jay Leno's garage. Jay, thanks again for bringing us here. Thank you guys for wondering where they're going. Catholics have the Vatican.",
  'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=0.0s'},
 {'start': 13.0,
  'end': 23.0,
  'text': "Jay, thanks again for bringing us here. Thank you guys for wondering where they're going. Catholics have the Vatican. I go, oh, what is it? It's totally different. I am in the wrong, oh, I see.",
  'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=13.0s'},
 {'start': 18.0,
  'end': 26.0,
  'text': "I go, oh, what is it? It's totally different. I am in the wrong, oh, I see. But you guys see, you make you comprise. Yeah, you know. That's why you live in the Vatican.",
  'id': 'https://www.youtube.com/watch?v=kSu7fNDKqvM&t=18.0s'},
 {'start': 21.0,
  'end': 29.0,
  'text': "But you guys see, you make you comp

In [71]:
# imports
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [72]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191


In [74]:
encoding = tiktoken.get_encoding("cl100k_base")
# should print [83, 1609, 5963, 374, 2294, 0]
encoding.encode("tiktoken is great!")

[83, 1609, 5963, 374, 2294, 0]

In [93]:
df = pd.DataFrame(transcript_new_data)
# noteiced a row with empty text, removing that
df=df[df.text.ne('')]
# encode , I might not need this step
df["n_tokens"] = df.text.apply(lambda x: len(encoding.encode(str(x))))
# filter too long text if any
df = df[df.n_tokens <= max_tokens]
df

,start,end,text,id,n_tokens
0,0.0,19.0,"So Catholics have the Vatican. Soccer fans, ol...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,52
1,13.0,23.0,"Jay, thanks again for bringing us here. Thank ...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,49
2,18.0,26.0,"I go, oh, what is it? It's totally different. ...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,49
3,21.0,29.0,"But you guys see, you make you comprise. Yeah,...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,51
4,25.0,34.0,"You know, you're in the Vatican. You know, you...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,56
...,...,...,...,...,...
155,1226.0,1245.0,"Cars don't have chrome anymore. Yeah, not to s...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,61
156,1232.0,1251.0,And I would use other polishes. I noticed on t...,https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,73
157,1241.0,1258.0,"So I said, let's find a company and develop a ...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,63
158,1249.0,1262.0,That's one that first took off. And then we bu...,https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,56


160

In [94]:
import openai
openai.api_key = 'sk-ADD YOUR OPENAI API KEY'
df["embedding"] = df.text.apply(lambda x: get_embedding(x, engine=embedding_model))
df[0:5]

,start,end,text,id,n_tokens,embedding
0,0.0,19.0,"So Catholics have the Vatican. Soccer fans, ol...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,52,"[0.023297294974327087, 0.0054927244782447815, ..."
1,13.0,23.0,"Jay, thanks again for bringing us here. Thank ...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,49,"[0.017099745571613312, -0.017125753685832024, ..."
2,18.0,26.0,"I go, oh, what is it? It's totally different. ...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,49,"[0.025255121290683746, -0.01768929697573185, 0..."
3,21.0,29.0,"But you guys see, you make you comprise. Yeah,...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,51,"[0.0219894889742136, -0.011526113376021385, 0...."
4,25.0,34.0,"You know, you're in the Vatican. You know, you...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,56,"[0.006469056010246277, -0.022417327389121056, ..."


In [95]:
df["embedding"] = df.embedding.apply(np.array)
df[0:5]

,start,end,text,id,n_tokens,embedding
0,0.0,19.0,"So Catholics have the Vatican. Soccer fans, ol...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,52,"[0.023297294974327087, 0.0054927244782447815, ..."
1,13.0,23.0,"Jay, thanks again for bringing us here. Thank ...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,49,"[0.017099745571613312, -0.017125753685832024, ..."
2,18.0,26.0,"I go, oh, what is it? It's totally different. ...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,49,"[0.025255121290683746, -0.01768929697573185, 0..."
3,21.0,29.0,"But you guys see, you make you comprise. Yeah,...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,51,"[0.0219894889742136, -0.011526113376021385, 0...."
4,25.0,34.0,"You know, you're in the Vatican. You know, you...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,56,"[0.006469056010246277, -0.022417327389121056, ..."
...,...,...,...,...,...,...
155,1226.0,1245.0,"Cars don't have chrome anymore. Yeah, not to s...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,61,"[-0.0001401206391165033, 0.01846638135612011, ..."
156,1232.0,1251.0,And I would use other polishes. I noticed on t...,https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,73,"[0.014332226477563381, 0.024213284254074097, 0..."
157,1241.0,1258.0,"So I said, let's find a company and develop a ...",https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,63,"[0.010685447603464127, 0.012360936962068081, -..."
158,1249.0,1262.0,That's one that first took off. And then we bu...,https://www.youtube.com/watch?v=kSu7fNDKqvM&t=...,56,"[0.005613747518509626, -0.02213779278099537, 0..."


In [96]:
query = 'Is EV the futuer ?'
query_embedding = get_embedding(query, engine=embedding_model)
query_embedding

[0.0028430733364075422,
 -0.01623658835887909,
 -0.004533549305051565,
 0.008545923978090286,
 -0.019577449187636375,
 0.0024622150231152773,
 -0.01571541279554367,
 -0.009514774195849895,
 -0.0026693486142903566,
 -0.005746282171458006,
 -0.008258610032498837,
 0.03533295542001724,
 -0.007075945381075144,
 0.00704253651201725,
 0.006080368533730507,
 0.014873516745865345,
 0.02238377369940281,
 -0.011018161661922932,
 0.02926594950258732,
 -0.008138339035212994,
 -0.014419158920645714,
 0.012013738974928856,
 0.014245434664189816,
 -0.021795781329274178,
 0.009949086233973503,
 0.0048375679180026054,
 0.010503669269382954,
 -0.024989645928144455,
 -0.007550347596406937,
 -0.0015376316150650382,
 0.019457178190350533,
 0.01655731163918972,
 0.005502399522811174,
 -0.025791453197598457,
 -0.01109166070818901,
 -0.01849501021206379,
 -0.0007232965435832739,
 0.004179418087005615,
 0.025150006636977196,
 -0.008218520320951939,
 0.02227686531841755,
 0.010503669269382954,
 -0.0029917417559

In [110]:
# find the most relevant parts of the video transcript to the query
df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))
results = (df.sort_values("similarity", ascending=False).head(3))
results = results.set_index(['id'])

In [116]:
from IPython.display import HTML
HTML(results[['start','similarity', 'text']].to_html(render_links=True, escape=False))

,start,similarity,text
id,,,
https://www.youtube.com/watch?v=kSu7fNDKqvM&t=897.0s,897.0,0.835220,"Is that, like you said, a taxpayer for trucks, spurred that growth. Are we going to see more EV adoption with that? Oh, sure. I mean, it took 75 years for every American home to have a telephone. It took five years for every American home to have a computer. So I think using that same sort of thing, you see how quickly it's moving now."
https://www.youtube.com/watch?v=kSu7fNDKqvM&t=892.0s,892.0,0.831535,"And I think that's the thing of that. But big, another extension of that taxpayer for EV. Right. Is that, like you said, a taxpayer for trucks, spurred that growth. Are we going to see more EV adoption with that? Oh, sure."
https://www.youtube.com/watch?v=kSu7fNDKqvM&t=483.0s,483.0,0.825912,"People would joke about electric. Oh, they're slow, whatever it might be. So I give him a lot of credit for it. I mean, I think the EV will be the savior of the classic car industry. Really? Yeah, because much like the automobile was the savior of the American horse."


References: https://lablab.ai/t/whisper-transcribe-youtube-video